# Etapa 3: Emparelhamento de Características (Matching)

**Objetivo:** Utilizar os keypoints e descritores extraídos na Etapa 2 para encontrar correspondências entre pares de imagens consecutivas.

**Pipeline:**
1.  **Carregar Dados:** Ler os arquivos `.npz` (keypoints/descritores) e `ordem_imagens.json` da Etapa 2.
2.  **Emparelhar (Matching):** Aplicar e comparar dois algoritmos:
    - **Brute-Force Matcher (BFMatcher):** Compara cada descritor da primeira imagem com todos os da segunda.
    - **FLANN (Fast Library for Approximate Nearest Neighbors):** Algoritmo otimizado para busca rápida em grandes conjuntos de dados.
3.  **Filtrar:** Aplicar o **Ratio Test de David Lowe** para remover correspondências ambíguas e manter apenas os matches de alta confiança.
4.  **Salvar Resultados:** Armazenar os matches filtrados em arquivos `.npz` para a Etapa 4 (Cálculo de Homografia).
5.  **Visualizar e Analisar:** Gerar imagens de visualização e uma tabela comparativa final.

In [43]:
import os
import glob
import json
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Tuple
from itertools import combinations 
from fnmatch import fnmatch        

## Funções Auxiliares (Carregamento, Salvamento e Visualização)

In [44]:
def desserializar_keypoints(arr: np.ndarray) -> List[cv2.KeyPoint]:
    """
    Reconstrói lista de cv2.KeyPoint a partir do array (N,7).
    """
    kps = []
    for row in arr:
        k = cv2.KeyPoint(
            x=float(row[0]), y=float(row[1]), size=float(row[2]),
            angle=float(row[3]), response=float(row[4]),
            octave=int(row[5]), class_id=int(row[6])
        )
        kps.append(k)
    return kps

def carregar_features_npz(caminho_npz: str) -> Tuple[List[cv2.KeyPoint], np.ndarray, str]:
    """
    Lê um .npz salvo pela Etapa 2 e retorna (keypoints, descriptors, imagem_absoluta).
    """
    with np.load(caminho_npz, allow_pickle=True) as data:
        kps = desserializar_keypoints(data["keypoints"])
        desc = data["descriptors"]
        img_path = str(data["imagem_absoluta"])
    return kps, desc, img_path

def salvar_matches_npz(caminho_saida: str, matches: List[cv2.DMatch]):
    """
    Converte uma lista de matches para um array (N,3) e salva em .npz.
    Colunas: (queryIdx, trainIdx, distance)
    """
    if not matches:
        data = np.zeros((0, 3), dtype=np.float32)
    else:
        data = np.array([[m.queryIdx, m.trainIdx, m.distance] for m in matches], dtype=np.float32)
    np.savez_compressed(caminho_saida, matches=data)

def desenhar_matches_custom(
    img1, kps1, img2, kps2, matches,
    num_matches_a_desenhar=25,
    cor_linha=(255, 0, 255), espessura_linha=3,
    cor_ponto=(0, 255, 0), raio_ponto=15
):
    """Desenha matches entre duas imagens com linhas e pontos customizáveis."""
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    output_img = np.zeros((max(h1, h2), w1 + w2, 3), dtype=np.uint8)
    output_img[:h1, :w1] = img1
    output_img[:h2, w1:] = img2

    if kps1 is None or kps2 is None:
        return output_img

    for match in matches[:num_matches_a_desenhar]:
        pt1_idx = match.queryIdx
        pt1 = (int(kps1[pt1_idx].pt[0]), int(kps1[pt1_idx].pt[1]))
        pt2_idx = match.trainIdx
        pt2 = (int(kps2[pt2_idx].pt[0] + w1), int(kps2[pt2_idx].pt[1]))
        cv2.line(output_img, pt1, pt2, cor_linha, espessura_linha)
        cv2.circle(output_img, pt1, raio_ponto, cor_ponto, thickness=-1)
        cv2.circle(output_img, pt2, raio_ponto, cor_ponto, thickness=-1)
        
    return output_img

## Funções de Emparelhamento (Matching) e Filtragem

In [45]:
def criar_matcher(nome_matcher: str, desc_dtype):
    """
    Cria um objeto matcher (BF ou FLANN) configurado corretamente.
    """
    nome_matcher = nome_matcher.lower()

    if nome_matcher == 'bf':
        norm_type = cv2.NORM_L2 if desc_dtype == np.float32 else cv2.NORM_HAMMING
        return cv2.BFMatcher(norm_type)

    if nome_matcher == 'flann':
        if desc_dtype == np.float32:  # Para SIFT
            FLANN_INDEX_KDTREE = 1
            index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        else:  # Para ORB/AKAZE (descritores binários)
            FLANN_INDEX_LSH = 6
            index_params = dict(
                algorithm=FLANN_INDEX_LSH,
                table_number=6, key_size=12, multi_probe_level=1
            )
        search_params = dict(checks=50)
        return cv2.FlannBasedMatcher(index_params, search_params)

    raise ValueError(f"Matcher '{nome_matcher}' não reconhecido.")


def filtrar_matches_ratio_test(matches_brutos, ratio=0.75):
    """
    Aplica o Ratio Test de David Lowe para filtrar matches ambíguos.
    Requer que os matches tenham sido encontrados com k=2.
    """
    matches_filtrados = []
    # Itera de forma segura, verificando o tamanho de cada resultado
    for match_pair in matches_brutos:
        # Garante que temos 2 vizinhos para desempacotar, evitando o ValueError
        if len(match_pair) == 2:
            m, n = match_pair  # Desempacota somente se for seguro
            if m.distance < ratio * n.distance:
                matches_filtrados.append(m)
    return matches_filtrados

def gerar_pares_indices(n_imagens: int, all_pairs: bool = False, wrap: bool = True):
    """
    Retorna lista de pares (i, j).
    - all_pairs=False: pares sequenciais
      * wrap=True  -> inclui (último, primeiro)
      * wrap=False -> só (0,1), (1,2), ..., (n-2, n-1)
    - all_pairs=True: todas as combinações i<j (sem wrap)
    """
    if n_imagens < 2:
        return []
    if all_pairs:
        return list(combinations(range(n_imagens), 2))  # NEW: todas as combinações i<j
    else:
        if wrap:
            return [(i, (i+1) % n_imagens) for i in range(n_imagens)]
        else:
            return [(i, i+1) for i in range(n_imagens - 1)]
def escolher_conjuntos(raiz: str,
                       only: list[str] | None = None,
                       include_patterns: list[str] | None = None,
                       exclude_patterns: list[str] | None = None) -> list[str]:
    """
    Lista subpastas de 'raiz' aplicando filtros.
    - only: lista explícita (se fornecida, tem prioridade)
    - include_patterns: padrões glob (ex.: ["2024_*", "scene*"])
    - exclude_patterns: padrões glob para excluir
    """
    todos = sorted([d for d in os.listdir(raiz) if os.path.isdir(os.path.join(raiz, d))])
    if only:
        base = [d for d in todos if d in set(only)]
    else:
        base = todos
        if include_patterns:
            base = [d for d in base if any(fnmatch(d, pat) for pat in include_patterns)]
    if exclude_patterns:
        base = [d for d in base if not any(fnmatch(d, pat) for pat in exclude_patterns)]
    return base

## Pipeline Principal da Etapa 3

In [50]:
def analisar_matches_conjunto(caminho_base_etapa2: str, matchers_a_usar: List[str], todos_os_pares: bool = False, salvar_apenas_imgs_de_pares_sequenciais: bool = True):
    """
    Pipeline da Etapa 3: Carrega features, realiza matching, filtra, salva e visualiza.
    Agora inclui o casamento do último com o primeiro (wrap-around).
    Agora pode casar pares sequenciais (com ou sem wrap) ou todas as combinações.
    """
    print(f"\n{'='*65}\nAnalisando Matches para o conjunto: '{os.path.basename(caminho_base_etapa2)}'\n{'='*65}")

    # Carrega a ordem das imagens
    caminho_json = os.path.join(caminho_base_etapa2, "ordem_imagens.json")
    try:
        with open(caminho_json, "r") as f:
            arquivos_de_imagem_abs = json.load(f)
    except FileNotFoundError:
        print(f"[Erro] 'ordem_imagens.json' não encontrado em '{caminho_base_etapa2}'. Abortando.")
        return
    
    # Itera sobre os detectores (sift, orb, etc.)
    detectores = [d for d in os.listdir(caminho_base_etapa2) if os.path.isdir(os.path.join(caminho_base_etapa2, d))]
    for nome_detector in detectores:
        print(f"\n--- Detector: {nome_detector.upper()} ---")
        caminho_detector = os.path.join(caminho_base_etapa2, nome_detector)
        
        # Carrega todos os arquivos de features para este detector
        arquivos_npz = sorted(glob.glob(os.path.join(caminho_detector, "*_features.npz")))
        n = len(arquivos_npz)
        if n < 2:
            continue
        pares = gerar_pares_indices(n, all_pairs=todos_os_pares, wrap=True)

        # Itera sobre os matchers (bf, flann)
        for nome_matcher in matchers_a_usar:
            print(f"  - Matcher: {nome_matcher.upper()}")
            
            # Cria o diretório de saída para esta combinação
            dir_saida = os.path.join("resultados_etapa3", os.path.basename(caminho_base_etapa2), nome_detector, nome_matcher)
            os.makedirs(dir_saida, exist_ok=True)

            # Itera sobre os pares de imagens, incluindo o último com o primeiro
            for (i, j) in pares:
                kps1, desc1, path1 = carregar_features_npz(arquivos_npz[i])
                kps2, desc2, path2 = carregar_features_npz(arquivos_npz[j])

                if desc1 is None or desc2 is None or desc1.shape[0] == 0 or desc2.shape[0] == 0:
                    salvar_matches_npz(os.path.join(dir_saida, f"matches_{i:03d}_{j:03d}.npz"), [])
                    continue

                matcher = criar_matcher(nome_matcher, desc1.dtype)
                matches_brutos = matcher.knnMatch(desc1, desc2, k=2)
                matches_filtrados = filtrar_matches_ratio_test(matches_brutos)
                
                # Salva os matches filtrados para a Etapa 4
                caminho_saida_npz = os.path.join(dir_saida, f"matches_{i:03d}_{j:03d}.npz")
                salvar_matches_npz(caminho_saida_npz, matches_filtrados)


                # Gera preview para TODOS os pares processados e salva como imagem por par
                # (antes o código só gerava o preview para o par 0-1 do SIFT em conjuntos específicos)
                nome_conjunto = os.path.basename(caminho_base_etapa2)
                try:
                    img1 = cv2.imread(path1)
                    img2 = cv2.imread(path2)
                    if img1 is None or img2 is None:
                        print(f"    [Aviso] Não foi possível carregar imagem(s) para preview: {path1}, {path2}")
                    else:
                        if not salvar_apenas_imgs_de_pares_sequenciais or j == (i + 1) % n:
                            print(f"    -> Par {i}-{j}: {len(matches_filtrados)} matches filtrados encontrados. Gerando preview...")
                            preview = desenhar_matches_custom(img1, kps1, img2, kps2, matches_filtrados)
                            caminho_saida_preview = os.path.join(dir_saida, f"preview_matches_filtrados_par_{i:03d}_{j:03d}.jpg")
                            cv2.imwrite(caminho_saida_preview, preview)
                        else:
                            print(f"    -> Par {i}-{j}: {len(matches_filtrados)} matches filtrados encontrados. Ignorando preview.")
                        # opcional: mostrar apenas quando necessário (comente se não quiser output no notebook)
                        # plt.figure(figsize=(20, 10))
                        # plt.imshow(cv2.cvtColor(preview, cv2.COLOR_BGR2RGB))
                        # plt.title(f'Matches Filtrados (Ratio Test) - {nome_detector.upper()}/{nome_matcher.upper()} - {len(matches_filtrados)} matches')
                        # plt.axis('off')
                        # plt.show()
                except Exception as e:
                    print(f"    [Erro] Falha ao gerar preview para o par {i}-{j}: {e}")


## Execução do Pipeline para os Conjuntos de Imagens

In [ ]:
RAIZ_ETAPA2 = "resultados_etapa2"
MATCHERS = ['bf', 'flann']

conjuntos = escolher_conjuntos(
    RAIZ_ETAPA2,
    only=None,  # Exemplo: ["images-normal", "images-pro"]
    include_patterns=None,  # Exemplo: ["images_developed*"]
    exclude_patterns=None   # Exemplo: ["images-developed*"]
)

for nome_conjunto in sorted(conjuntos):
    caminho_do_conjunto = os.path.join(RAIZ_ETAPA2, nome_conjunto)
    analisar_matches_conjunto(caminho_do_conjunto, MATCHERS, todos_os_pares=False) #ATENÇÃO, todos_os_pares=True APENAS PARA CALCULO DO GRAFO


Analisando Matches para o conjunto: 'images-developed-png-16bit'

--- Detector: ORB ---
  - Matcher: BF
    -> Par 0-1: 148 matches filtrados encontrados. Gerando preview...
    -> Par 0-1: 148 matches filtrados encontrados. Gerando preview...
    -> Par 0-2: 53 matches filtrados encontrados. Ignorando preview.
    -> Par 0-2: 53 matches filtrados encontrados. Ignorando preview.
    -> Par 0-3: 22 matches filtrados encontrados. Ignorando preview.
    -> Par 0-3: 22 matches filtrados encontrados. Ignorando preview.
    -> Par 0-4: 26 matches filtrados encontrados. Ignorando preview.
    -> Par 0-4: 26 matches filtrados encontrados. Ignorando preview.
    -> Par 0-5: 10 matches filtrados encontrados. Ignorando preview.
    -> Par 0-5: 10 matches filtrados encontrados. Ignorando preview.
    -> Par 0-6: 6 matches filtrados encontrados. Ignorando preview.
    -> Par 0-6: 6 matches filtrados encontrados. Ignorando preview.
    -> Par 0-7: 5 matches filtrados encontrados. Ignorando preview.

## Geração do Relatório Comparativo Final

In [ ]:
def gerar_relatorio_final_etapa3(raiz_resultados: str, only_sequential: bool = True): #only sequential=True só é útil quando calculando os pesos para o grafo
    """Gera uma tabela CSV comparando a média de matches por combinação."""
    print("\nGerando relatório comparativo final da Etapa 3...")
    dados_tabela = []
    
    # Navega pela estrutura de pastas gerada
    for nome_conjunto in sorted(os.listdir(raiz_resultados)):
        caminho_conjunto = os.path.join(raiz_resultados, nome_conjunto)
        if not os.path.isdir(caminho_conjunto): continue
            
        for nome_detector in sorted(os.listdir(caminho_conjunto)):
            caminho_detector = os.path.join(caminho_conjunto, nome_detector)
            if not os.path.isdir(caminho_detector): continue
                
            for nome_matcher in sorted(os.listdir(caminho_detector)):
                caminho_matcher = os.path.join(caminho_detector, nome_matcher)
                if not os.path.isdir(caminho_matcher): continue
                    
                arquivos_matches_npz = glob.glob(os.path.join(caminho_matcher, "matches_*.npz"))
                # NEW: montar pares e, se solicitado, filtrar só sequenciais com wrap
                pares = []
                idxs = set()
                for npz_path in arquivos_matches_npz:
                    base = os.path.splitext(os.path.basename(npz_path))[0]  # matches_000_001
                    _, si, sj = base.split("_")
                    i, j = int(si), int(sj)
                    pares.append((i, j, npz_path))
                    idxs.add(i); idxs.add(j)

                if only_sequential and idxs:
                    n = max(idxs) + 1
                    seq = {(k, (k + 1) % n) for k in range(n)}  # pares sequenciais com wrap
                    pares = [t for t in pares if (t[0], t[1]) in seq]
                    pares.sort(key=lambda t: (t[0], t[1]))  # ordem estável

                if not pares:
                    media_matches = 0.0
                else:
                    contagens = []
                    for _, _, npz in pares:
                        with np.load(npz) as data:
                            contagens.append(len(data['matches']))
                    media_matches = np.mean(contagens)
                
                dados_tabela.append({
                    "conjunto": nome_conjunto,
                    "detector": nome_detector,
                    "matcher": nome_matcher,
                    "media_matches_filtrados_por_par": media_matches
                })
    
    if not dados_tabela:
        print("Nenhum dado encontrado para gerar o relatório.")
        return

    df_final = pd.DataFrame(dados_tabela)
    df_final['media_matches_filtrados_por_par'] = df_final['media_matches_filtrados_por_par'].map('{:,.2f}'.format)
    
    print(f"\n{'='*80}\nRELATÓRIO COMPARATIVO FINAL - ETAPA 3\n{'='*80}")
    
    caminho_saida_final = os.path.join(raiz_resultados, "relatorio_comparativo_matches.csv")
    df_final.to_csv(caminho_saida_final, index=False, sep=';')
    print(f"\nTabela resumo salva em: '{caminho_saida_final}'")
    return df_final

# Execução
gerar_relatorio_final_etapa3("resultados_etapa3", only_sequential=True)


Gerando relatório comparativo final da Etapa 3...

RELATÓRIO COMPARATIVO FINAL - ETAPA 3

Tabela resumo salva em: 'resultados_etapa3/relatorio_comparativo_matches.csv'


,conjunto,detector,matcher,media_matches_filtrados_por_par
0,images-developed-png-16bit,akaze,bf,297.97
1,images-developed-png-16bit,akaze,flann,303.59
2,images-developed-png-16bit,orb,bf,117.83
3,images-developed-png-16bit,orb,flann,126.00
4,images-developed-png-16bit,sift,bf,166.62
5,images-developed-png-16bit,sift,flann,167.76
6,images-developed-png-8bit,akaze,bf,295.31
7,images-developed-png-8bit,akaze,flann,300.45
8,images-developed-png-8bit,orb,bf,118.55
9,images-developed-png-8bit,orb,flann,127.03
